In [4]:
import numpy as np

# translation matrix
trans = np.array(
    [[1, 0, 0, 1],
     [0, 1, 0, 2],
     [0, 0, 1, 3],
     [0, 0, 0, 1]]
)

# rotation matrix
rot = np.array(
    [[ 0.4198, -0.3450, -0.8395, 0.0000],
     [-0.2159,  0.8605, -0.4615, 0.0000],
     [ 0.8816,  0.3749,  0.2867, 0.0000],
     [ 0.0000,  0.0000,  0.0000, 1.0000]]
)

# rototranslation matrix
tr = trans @ rot
rt = rot @ trans
print(tr)
print(rt)

[[ 0.4198 -0.345  -0.8395  1.    ]
 [-0.2159  0.8605 -0.4615  2.    ]
 [ 0.8816  0.3749  0.2867  3.    ]
 [ 0.      0.      0.      1.    ]]
[[ 0.4198 -0.345  -0.8395 -2.7887]
 [-0.2159  0.8605 -0.4615  0.1206]
 [ 0.8816  0.3749  0.2867  2.4915]
 [ 0.      0.      0.      1.    ]]


If we translate first, then rotate, the matrix construction is trivial. Since translating the scene is equivalent to -1 * translation of the camera, this holds for either strategy.